# Mapping the socio-economic situation in Mpumalanga, South Africa

In order to meet its commitments to the Paris agreements, South Africa is investigating ways to ween itself off of coal. This will be felt especially strongly in Mpumalanga, South Africa, a province in the north-east of the country between Johannesburg and Eswatini. Mpumalanga is a sparsely populated area home to most of South Africa's coal communities and coal power plants.

In [ ]:
import sys
import os
import rasterio

import pandas as pd
import geopandas as gpd
import skimage.graph as graph

from shapely.geometry import box, Point

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTrocks.dataMisc as dataMisc
import GOSTrocks.ntlMisc as ntlMisc
import GOSTrocks.ghslMisc as ghslMisc
import GOSTrocks.rasterMisc as rMisc
import GOSTrocks.mapMisc as mapMisc
from GOSTrocks.misc import tPrint

sys.path.insert(0, "/home/wb411133/Code/GOSTNets_Raster/src")
import GOSTNetsRaster.market_access as ma

%load_ext autoreload
%autoreload 2

In [ ]:
# Define input variables
in_folder = "/home/wb411133/projects/URB_SURDR_ZAF"
ntl_folder = os.path.join(in_folder, "NTL_data")
ghsl_folder = os.path.join(in_folder, "GHSL_data")
urban_folder = os.path.join(in_folder, "URBAN")
ma_folder = os.path.join(in_folder, "market_access")
infra_folder = os.path.join(in_folder, "Infra")
protected_areas_folder = os.path.join(in_folder, "Protected_Areas")
for f in [
    in_folder,
    ntl_folder,
    ghsl_folder,
    ma_folder,
    infra_folder,
    protected_areas_folder,
]:
    if not os.path.exists(f):
        os.makedirs(f)

# Define global variables
global_bounds = (
    "/home/public/Data/GLOBAL/ADMIN/ADMIN2/HighRes_20230328/shp/WB_GAD_ADM0.shp"
)
ghs_folder = "/home/public/Data/GLOBAL/GHSL"
ghs_built_folder = os.path.join(ghs_folder, "Built")
ghs_built_files = [x for x in os.listdir(ghs_built_folder) if x.endswith(".tif")]
ghs_smod_file = os.path.join(
    ghs_folder, "SMOD", "GHS_SMOD_E2020_GLOBE_R2023A_54009_1000_V1_0.tif"
)
ghs_ucdb = os.path.join(
    ghs_folder, "GHS_UCBD_R2019A", "GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg"
)
global_friction = "/home/public/Data/GLOBAL/INFRA/FRICTION_2020/2020_motorized_friction_surface.geotiff"
global_airports_file = os.path.join(
    infra_folder, "airport_volume_airport_locations.csv"
)

# Define local variables
admin0_file = os.path.join(in_folder, "ZAF_select_adm0.shp")
admin3_file = os.path.join(in_folder, "ADMIN", "admin3_geoBounds_FINAL.shp")
ghsl_thresh = 0.1
local_ghsl_file = os.path.join(in_folder, f"ghsl_combined_{int(ghsl_thresh*100)}.tif")
urban_raster = os.path.join(urban_folder, "zaf1k_cpo20_urban.tif")
urban_raster_pop = os.path.join(urban_folder, "zaf1k_cpo20.tif")
urban_extents_file = os.path.join(urban_folder, "cpo20_urban_extents.shp")
local_ghs_smod_file = os.path.join(in_folder, "GHS_SMOD_2020.tif")
major_urban_extents = os.path.join(in_folder, "major_cities_UCDB2019.shp")
local_friction_file = os.path.join(ma_folder, "friction_2020.tif")
local_airports = os.path.join(infra_folder, "airports_ZAF_and_neighbours.geojson")
local_ports = os.path.join(infra_folder, "Ports.shp")
tourist_locations = os.path.join(infra_folder, "Kruger_EntryGates.shp")
protected_areas = os.path.join(protected_areas_folder, "SAPAD_IR_2023_Q2_01.shp")

In [ ]:
# read in base data
ntl_files = dataMisc.aws_search_ntl()
admin0_bounds = gpd.read_file(global_bounds)
if not os.path.exists(admin0_file):
    zaf_bounds = admin0_bounds.loc[admin0_bounds["WB_A3"] == "ZAF"]
    zaf_bounds.to_file(admin0_file)
else:
    zaf_bounds = gpd.read_file(admin0_file)
neighbours = admin0_bounds.loc[
    admin0_bounds.intersects(zaf_bounds.unary_union.buffer(0.1))
]
#
admin1_bounds = dataMisc.get_geoboundaries("ZAF", "ADM1")
admin2_bounds = dataMisc.get_geoboundaries("ZAF", "ADM2")
admin3_bounds = dataMisc.get_geoboundaries("ZAF", "ADM3")
focal_state = admin1_bounds.loc[admin1_bounds["shapeName"] == "Mpumalanga"]

In [ ]:
admin1_bounds.to_file(os.path.join(in_folder, "admin1_geoBounds.shp"))
admin2_bounds.to_file(os.path.join(in_folder, "admin2_geoBounds.shp"))
admin3_bounds.to_file(os.path.join(in_folder, "admin3_geoBounds.shp"))

# Clip raster datasets

In [ ]:
# Clip out nighttime lights annual images
# Mpumalanga
ntlMisc.generate_annual_composites(
    focal_state.unary_union, out_folder=os.path.join(ntl_folder, "Mpumalanga")
)
# ZAF
# ntlMisc.generate_annual_composites(zaf_bounds.unary_union, out_folder=os.path.join(ntl_folder, "ZAF"))
# Neighbours
# ntlMisc.generate_annual_composites(neighbours.unary_union, out_folder=os.path.join(ntl_folder, "Neighbours"))

In [ ]:
# Clip out GHSL layers
for cur_raster_file in ghs_built_files:
    out_file = os.path.join(ghsl_folder, os.path.basename(cur_raster_file))
    if not os.path.exists(out_file):
        rMisc.clipRaster(
            rasterio.open(os.path.join(ghs_built_folder, cur_raster_file)),
            zaf_bounds,
            out_file,
            crop=False,
        )
        tPrint(out_file)

# Combine GHSL layers into single file
ghsl_files = sorted(
    [
        os.path.join(ghsl_folder, x)
        for x in os.listdir(ghsl_folder)
        if x.endswith(".tif")
    ]
)
if not os.path.exists(local_ghsl_file):
    ghsl_res = ghslMisc.combine_ghsl_annual(
        ghsl_files, built_thresh=ghsl_thresh, out_file=out_file
    )

In [ ]:
# clip out GHS-SMOD data
if not os.path.exists(local_ghs_smod_file):
    rMisc.clipRaster(
        rasterio.open(ghs_smod_file), neighbours, local_ghs_smod_file, crop=False
    )

In [ ]:
# Convert urban centres from the constrained world_pop 2020 dataset to vector
if not os.path.exists(urban_extents_file):
    urban_extents = rMisc.vectorize_raster(rasterio.open(urban_raster), bad_vals=[0])
    urban_extents["geometry"] = urban_extents["geometry"].apply(lambda x: x.buffer(0))

    # Attribute with population
    res = rMisc.zonalStats(urban_extents, urban_raster_pop, minVal=0)
    res = pd.DataFrame(res, columns=["SUM", "MIN", "MAX", "MEAN"])
    urban_extents["Pop2020"] = res["SUM"]
    urban_extents.to_file(urban_extents_file)

In [ ]:
# Extract major settlements from UCDB
if not os.path.exists(major_urban_extents):
    all_extents = gpd.read_file(ghs_ucdb)
    sel_extents = all_extents.loc[all_extents.intersects(box(*neighbours.total_bounds))]
    sel_extents.to_file(major_urban_extents)

In [ ]:
all_extents.head()

# Extract international airports

In [ ]:
if not os.path.exists(local_airports):
    inA = pd.read_csv(global_airports_file)
    inA_geom = [
        Point(x) for x in zip(inA["Airport1Longitude"], inA["Airport1Latitude"])
    ]
    inA = gpd.GeoDataFrame(inA, geometry=inA_geom, crs=4326)
    selA = inA.loc[inA.intersects(neighbours.unary_union)]
    selA.to_file(local_airports, driver="GeoJSON")

"""headers = {'Accept': 'application/json'}
ddh_international_airports = "https://wiki.worldbank.org/pages/viewpage.action?spaceKey=GEOS&title=Guide+to+procurement+of+satellite+imagery+and+derived+products"
ddh_r = requests.get(ddh_international_airports, headers=headers)
ddh_r.json()"""

# Market access

In [ ]:
if not os.path.exists(local_friction_file):
    rMisc.clipRaster(
        rasterio.open(global_friction), neighbours, local_friction_file, crop=False
    )

friction = rasterio.open(local_friction_file)
frictionD = friction.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
# Measure access to all major cities in ucdb
tt_major_cities = os.path.join(ma_folder, "tt_major_cities.tif")
if not os.path.exists(tt_major_cities):
    dests = gpd.read_file(major_urban_extents)
    dests["geometry"] = dests["geometry"].apply(lambda x: x.centroid)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, dests)
    with rasterio.open(tt_major_cities, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to international airports
tt_airports = os.path.join(ma_folder, "tt_airports.tif")
if not os.path.exists(tt_airports):
    airports = gpd.read_file(local_airports)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, airports)
    with rasterio.open(tt_airports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to international ports
tt_ports = os.path.join(ma_folder, "tt_ports.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(local_ports)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to kruger national park
tt_ports = os.path.join(ma_folder, "tt_tourism.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(tourist_locations)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
# Measure access to all national protected areas
tt_ports = os.path.join(ma_folder, "tt_protected.tif")
if not os.path.exists(tt_ports):
    ports = gpd.read_file(protected_areas)
    ports["geometry"] = ports["geometry"].apply(lambda x: x.centroid)
    travel_costs, traceback = ma.calculate_travel_time(friction, mcp, ports)
    with rasterio.open(tt_ports, "w", **friction.profile.copy()) as out_tt:
        out_tt.write_band(1, travel_costs)

In [ ]:
dests = gpd.read_file(admin3_file)
dests["geometry"] = dests["geometry"].apply(lambda x: x.centroid)

od_res = ma.calculate_od_matrix(friction, mcp, dests)
final_od = pd.DataFrame(od_res)
final_od.to_csv(os.path.join(ma_folder, "admins3_od.csv"))

In [ ]:
# calculate total population and nighttime lights brightness for each admin3
dests = gpd.read_file(admin3_file)
pop_res = rMisc.zonalStats(dests, urban_raster_pop, minVal=0, reProj=True)
pop_res = pd.DataFrame(pop_res, columns=["SUM", "MIN", "MAX", "MEAN"])
pop_res

In [ ]:
ntl_raster = os.path.join(ntl_folder, "Neighbours", "VIIRS_2022_annual.tif")
ntl_res = rMisc.zonalStats(dests, ntl_raster, minVal=1, reProj=True)
ntl_res = pd.DataFrame(ntl_res, columns=["SUM", "MIN", "MAX", "MEAN"])

In [ ]:
# Map random
dests["Pop"] = pop_res["SUM"]
dests["NTL"] = pop_res["SUM"]
mapMisc.static_map_vector(
    dests,
    "Pop",
    legend_loc="upper left",
    thresh=[0, 50000, 100000, 250000, 500000, 200000000],
)

# Calculate Main Place to Protected Areas OD

In [ ]:
municipalities = os.path.join(
    in_folder, "MiningCommunities", "MainPlaces", "MP_SA_2011.shp"
)
inM = gpd.read_file(municipalities)
inM["geometry"] = inM["geometry"].apply(lambda x: x.centroid)

destinations = gpd.read_file(protected_areas)
destinations = destinations.to_crs(22293)
destinations["area_km"] = destinations["geometry"].apply(lambda x: x.area / 1000000)
destinations = destinations.to_crs(4326)
destinations["geometry"] = destinations["geometry"].apply(lambda x: x.centroid)

In [ ]:
# Calculate travel time
popR = rasterio.open(urban_raster_pop)
ttr = rasterio.open(local_friction_file)
frictionD = ttr.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
od = ma.calculate_od_matrix(ttr, mcp, inM, destinations)
xx = pd.DataFrame(od)
xx.columns = inM.MP_CODE_st
xx.index = destinations.WDPAID
xx.to_csv(os.path.join(out_varun_folder, "mp_protected_areas_OD.csv"))

## Calculate Main Place OD

In [ ]:
municipalities = os.path.join(
    in_folder, "MiningCommunities", "MainPlaces", "MP_SA_2011.shp"
)
inM = gpd.read_file(municipalities)
inM["geometry"] = inM["geometry"].apply(lambda x: x.centroid)

In [ ]:
# Calculate travel time
popR = rasterio.open(urban_raster_pop)
ttr = rasterio.open(local_friction_file)
frictionD = ttr.read()[0, :, :] * 1000
mcp = graph.MCP_Geometric(frictionD)

In [ ]:
od = ma.calculate_od_matrix(ttr, mcp, inM, inM)
xx = pd.DataFrame(od)
xx.columns = inM.MP_CODE_st
xx.index = sel_sp.MP_CODE_st
xx.to_csv(os.path.join(out_varun_folder, "all_mp.csv"))

In [ ]:
xx.columns = inM.MP_CODE_st
xx.index = inM.MP_CODE_st

In [ ]:
xx.to_csv(os.path.join(ma_folder, "all_mp_od.csv"))

In [ ]:
xx.head()

# Calculate gravity

In [ ]:
ma.calculate_gravity?

In [ ]:
weights = pd.read_csv(os.path.join(in_folder, "ZONAL_RES", "named_places_zonal.csv"))
weights.head()

In [ ]:
weights.shape

In [ ]:
xx.shape

In [ ]:
simple_gravity = ma.calculate_gravity(xx)
simple_gravity.head()

In [ ]:
pop_gravity = ma.calculate_gravity(xx, weights["POP"].values, weights["POP"].values)
pop_gravity.head()

In [ ]:
ntl_gravity = ma.calculate_gravity(
    xx, weights["NTL_2023"].values, weights["NTL_2023"].values
)
ntl_gravity.head()

In [ ]:
inM = gpd.read_file(municipalities)

In [ ]:
def create_ma_geometry(ma_df, out_file, xx_inM, ma_col=["d_0.001"], driver="GeoJSON"):
    # create output geospatial market access data
    simple_geog = ma_df.copy()
    simple_geog = simple_geog.loc[:, ma_col]
    simple_geog["geometry"] = xx_inM["geometry"].values
    simple_geog = gpd.GeoDataFrame(simple_geog, geometry="geometry", crs=xx_inM.crs)
    pd.DataFrame(simple_geog.drop(["geometry"], axis=1)).to_csv(f"{out_file}.csv")
    simple_geog.to_file(out_file, driver=driver)
    return simple_geog

In [ ]:
ma_folder

In [ ]:
create_ma_geometry(
    simple_gravity,
    os.path.join(ma_folder, "simple_ma.shp"),
    inM,
    driver="ESRI Shapefile",
)
create_ma_geometry(
    pop_gravity, os.path.join(ma_folder, "pop_ma.shp"), inM, driver="ESRI Shapefile"
)
create_ma_geometry(
    ntl_gravity, os.path.join(ma_folder, "ntl_ma.shp"), inM, driver="ESRI Shapefile"
)

In [ ]:
weights.head()

In [ ]:
weights.loc[weights["MP_CODE_st"] == 798020]

# Debugging